In [18]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import load_img
import os

In [19]:
src_dir = '/kaggle/input/rice-image-classification-dataset/Rice_Image_Dataset'

In [20]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2,
    rotation_range=25,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2
)

validation_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2
)

In [21]:
import os

base_path = "/kaggle/input"
for root, dirs, files in os.walk(base_path):
    for d in dirs:
        print(os.path.join(root, d))

/kaggle/input/rice-image-classification-dataset
/kaggle/input/rice-image-classification-dataset/Rice_Image_Dataset
/kaggle/input/rice-image-classification-dataset/Rice_Image_Dataset/Karacadag
/kaggle/input/rice-image-classification-dataset/Rice_Image_Dataset/Basmati
/kaggle/input/rice-image-classification-dataset/Rice_Image_Dataset/Jasmine
/kaggle/input/rice-image-classification-dataset/Rice_Image_Dataset/Arborio
/kaggle/input/rice-image-classification-dataset/Rice_Image_Dataset/Ipsala


In [23]:
train_generator = train_datagen.flow_from_directory(
    src_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    seed=1
)

validation_generator = validation_datagen.flow_from_directory(
    src_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle=False,
    seed=1
)

Found 60000 images belonging to 5 classes.
Found 15000 images belonging to 5 classes.


In [24]:
base_model = VGG16(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
base_model.trainable = False

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [25]:
inputs = tf.keras.Input(shape=(224, 224, 3))
x = preprocess_input(inputs)
x = base_model(x, training=False)

x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)

outputs = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs, outputs)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ get_item (GetItem)        │ (None, 224, 224)       │              0 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ get_item_1 (GetItem)      │ (None, 224, 224)       │              0 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ get_item_2 (GetItem)      │ (None, 224, 224)       │              0 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stack (Stack)             │ (None, 224, 224, 3)    │              0 │ get_item[0][0],        │
│                           │                        │                │ get_item_1[0][0],      │
│                           │                        │                │ get_item_2[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add (Add)                 │ (None, 224, 224, 3)    │              0 │ stack[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ vgg16 (Functional)        │ (None, 7, 7, 512)      │     14,714,688 │ add[0][0]              │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling2d  │ (None, 512)            │              0 │ vgg16[0][0]            │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 512)            │              0 │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 128)            │         65,664 │ dropout[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 128)            │              0 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 5)              │            645 │ dropout_1[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 14,780,997 (56.39 MB)

 Trainable params: 66,309 (259.02 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [26]:
checkpoint_cb_all = tf.keras.callbacks.ModelCheckpoint(
    filepath='/kaggle/working/model_vgg16_epoch_{epoch:02d}_valLoss_{val_loss:.4f}.keras',
    save_best_only=False,  
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    verbose=1  
)

checkpoint_cb_best = tf.keras.callbacks.ModelCheckpoint(
    filepath='/kaggle/working/model_vgg16_best.keras',
    save_best_only=True,
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    verbose=1  
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    mode='min',
    patience=3,
    factor=0.5,
    min_lr=1e-6,
    verbose=1  
)

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    mode='min',
    patience=7,
    restore_best_weights=True,
    verbose=1  
)


In [28]:
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator,
    callbacks=[checkpoint_cb_all, checkpoint_cb_best, reduce_lr, early_stopping],
    verbose=1
)

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 460ms/step - accuracy: 0.4993 - loss: 2.5907
Epoch 1: saving model to /kaggle/working/model_vgg16_epoch_01_valLoss_0.2603.keras

Epoch 1: val_loss improved from inf to 0.26031, saving model to /kaggle/working/model_vgg16_best.keras
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 968s 506ms/step - accuracy: 0.4994 - loss: 2.5901 - val_accuracy: 0.9232 - val_loss: 0.2603 - learning_rate: 1.0000e-04
Epoch 2/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 386ms/step - accuracy: 0.7871 - loss: 0.5779
Epoch 2: saving model to /kaggle/working/model_vgg16_epoch_02_valLoss_0.2195.keras

Epoch 2: val_loss improved from 0.26031 to 0.21949, saving model to /kaggle/working/model_vgg16_best.keras
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 785s 417ms/step - accuracy: 0.7871 - loss: 0.5778 - val_accuracy: 0.9359 - val_loss: 0.2195 - learning_rate: 1.0000e-04
Epoch 3/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 391ms/step - accuracy: 0.8389 - loss: 0.4215
Epoch 3: saving model to /kaggle/working/model_vgg16_epoch_03

In [29]:
from tensorflow.keras.models import load_model

best_model_path = "/kaggle/working/model_vgg16_best.keras"
model = load_model(best_model_path)
print("Best model loaded successfully!")

Best model loaded successfully!


In [30]:
final_model_path = "/kaggle/working/model_vgg16_final.keras"
model.save(final_model_path)
print("Best model saved as final .keras model")

Best model saved as final .keras model


In [31]:
loss, accuracy = model.evaluate(validation_generator)
print(f"Final Model Accuracy: {accuracy * 100:.2f}%")
print(f"Final Model Loss: {loss:.4f}")

469/469 ━━━━━━━━━━━━━━━━━━━━ 61s 128ms/step - accuracy: 0.9867 - loss: 0.0534
Final Model Accuracy: 97.29%
Final Model Loss: 0.0837
